### Long Context Handling Capability

This notebook illustrates how you can use `capabilities` to give any `Conversable` agent an ability to handle long contexts.

In [1]:
## Uncomment to install pyautogen if you don't have it already
#! pip install pyautogen

In [2]:
import autogen
from autogen.agentchat.contrib.capabilities.context_handling import LongContextCapability

In [4]:
config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list,
    },
)
long_context_capability = LongContextCapability(max_messages=4)
long_context_capability.add_to_agent(assistant)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    max_consecutive_auto_reply=10,
)

user_proxy.initiate_chat(assistant, message="plot and save a graph of x^2 from -10 to 10")

user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------
Truncated 0:
[{'content': 'plot and save a graph of x^2 from -10 to 10', 'role': 'user'}]
assistant (to user_proxy):

To plot and save a graph of the function x^2 from -10 to 10, we can use Python with libraries such as matplotlib. First, I will provide you with the code to perform this task. Then you will need to run this code on your machine. The code will generate and save the plot in the current directory as an image file.

Please execute the following Python code:

```python
# filename: plot_x_squared.py
import matplotlib.pyplot as plt
import numpy as np

# Create an array of x values from -10 to 10
x = np.linspace(-10, 10, 400)
# Calculate the square of x values
y = x ** 2

# Create the plot
plt.figure(figsize=(8, 6))
plt.plot(x, y, '-r', label='y = x^2')
plt.title('Graph of y = x^2')
plt.xlabel('x', fontsize=14)
plt.ylabel('y',